In [1]:
import numpy as np
import vtk
import xarray as xr

In [2]:
#データ出力
# writer = vtkXMLDataObjectWriter()
# writer.SetFileName("tmp.vts")
# writer.SetInputData(grid)
# writer.Update()

In [2]:
zcor = 10

# 3D vtk

In [3]:

def make3Dvtk(ds, outfilename, zcor):
    dsp = ds.isel(time=-1) 

    points = vtk.vtkPoints()

    WL = dsp['WL'].values
    dep = dsp['WL'].values - dsp['zb'].values
    X = dsp.xz.values
    Y = dsp.yz.values

    for sig in dsp['sigma'].values:
        for j in range(dsp.dims.mapping['y']):
            for i in range(dsp.dims.mapping['x']):
                z = (WL[i,j]+dep[i,j]*sig)*zcor
                points.InsertNextPoint(X[i,j], Y[i,j], z)

    #点データをstructuredGridにセットする
    structured_mesh = vtk.vtkStructuredGrid()
    structured_mesh.SetExtent(0, dsp.dims.mapping['x']-1, 0, dsp.dims.mapping['y']-1, 0, dsp.dims.mapping['sigma']-1)
    structured_mesh.SetPoints(points)

    # velocity
    arr = vtk.vtkDoubleArray()
    arr.SetNumberOfComponents(3) 
    arr.SetNumberOfTuples((dsp.dims.mapping['x'])*(dsp.dims.mapping['y'])*(dsp.dims.mapping['sigma']))
    arr.SetName("velocity")

    id = 0
    for k in range(dsp.dims.mapping['sigma']):
        for j in range(dsp.dims.mapping['y']):
            for i in range(dsp.dims.mapping['x']):
                u = dsp['VxyX'].values[k,i,j]
                v = dsp['VxyY'].values[k,i,j]
                w = dsp['W'].values[k,i,j]
                # arr.InsertNextValue(u)
                # arr.InsertNextTuple((u,v,w))
                arr.SetTuple(id, [u,v,w])
                id += 1

    #StructuredGridのPointDataに温度データを追加
    structured_mesh.GetPointData().AddArray(arr)

    # # concentaration
    # arr2 = vtk.vtkDoubleArray()
    # # arr2.SetNumberOfComponents(1) 
    # # arr2.SetNumberOfTuples((dsp.dims.mapping['x'])*(dsp.dims.mapping['y'])*(dsp.dims.mapping['sigma']))
    # arr2.SetName("concetration")

    # C = dsp['C'].values
    # for k in range(dsp.dims.mapping['sigma']):
    #     for j in range(dsp.dims.mapping['y']):
    #         for i in range(dsp.dims.mapping['x']):
    #             arr2.InsertNextValue(C[k,i,j])

    # #StructuredGridのPointDataに温度データを追加
    # structured_mesh.GetPointData().AddArray(arr2)

    writer = vtk.vtkXMLDataObjectWriter()
    writer.SetFileName(outfilename)     #出力ファイル名
    # writer.SetFileName("tmp01.vts")
    writer.SetInputData(structured_mesh)
    writer.Update()


ds = xr.open_dataset('../out_cellcenter01.nc')
d = make3Dvtk(ds, 'flowprof01.vts', zcor)

ds = xr.open_dataset('../out_cellcenter02.nc')
d = make3Dvtk(ds, 'flowprof02.vts', zcor)

# 2D vtk

In [4]:
def make2Dvtk(ds, outWLname, outZbname):
    dsp = ds.isel(time=-1)

    points = vtk.vtkPoints()
    points2 = vtk.vtkPoints()

    WL = dsp['WL'].values
    zb = dsp['zb'].values
    X = dsp.xz.values
    Y = dsp.yz.values

    for j in range(dsp.dims.mapping['y']):
        for i in range(dsp.dims.mapping['x']):
            points.InsertNextPoint(X[i,j], Y[i,j], WL[i,j]*zcor)
            points2.InsertNextPoint(X[i,j], Y[i,j], zb[i,j]*zcor)

#点データをstructuredGridにセットする
    structured_mesh = vtk.vtkStructuredGrid()
    structured_mesh.SetExtent(0, dsp.dims.mapping['x']-1, 0, dsp.dims.mapping['y']-1, 0, 0)
    structured_mesh.SetPoints(points)

    writer = vtk.vtkXMLDataObjectWriter()
    writer.SetFileName(outWLname)
    writer.SetInputData(structured_mesh)
    writer.Update()

    structured_mesh2 = vtk.vtkStructuredGrid()
    structured_mesh2.SetExtent(0, dsp.dims.mapping['x']-1, 0, dsp.dims.mapping['y']-1, 0, 0)
    structured_mesh2.SetPoints(points2)

    writer2 = vtk.vtkXMLDataObjectWriter()
    writer2.SetFileName(outZbname)
    writer2.SetInputData(structured_mesh2)
    writer2.Update()

ds = xr.open_dataset('../out_cellcenter01.nc')
d = make2Dvtk(ds,"WL01.vts", "Zb01.vts")

ds = xr.open_dataset('../out_cellcenter02.nc')
d = make2Dvtk(ds, "WL02.vts", "Zb02.vts")

In [10]:
points = vtk.vtkPoints()

# it = 8
# dsp = ds.isel(time=it)
WL = dsp['WL'].values
dep = dsp['WL'].values - dsp['zb'].values
X = dsp.xz.values
Y = dsp.yz.values

Xt, Yt, Zt = [], [], []
for i in range(dsp.dims.mapping['x']):
    for j in range(dsp.dims.mapping['y']):
        for sig in dsp['sigma'].values:
            z = WL[i,j]+dep[i,j]*sig*zcor
            points.InsertNextPoint(X[i,j], Y[i,j], z)
            # Xt.append(X[i,j])
            # Yt.append(Y[i,j])
            # Zt.append(z*100)

#点データをstructuredGridにセットする
structured_mesh = vtk.vtkStructuredGrid()
structured_mesh.SetExtent(0, dsp.dims.mapping['x']-1, 0, dsp.dims.mapping['y']-1, 0, dsp.dims.mapping['sigma']-1)
structured_mesh.SetPoints(points)

arr = vtk.vtkDoubleArray()
arr.SetNumberOfComponents(3) 
arr.SetNumberOfTuples((dsp.dims.mapping['x'])*(dsp.dims.mapping['y'])*(dsp.dims.mapping['sigma']))
arr.SetName("velocity")

id = 0
for i in range(dsp.dims.mapping['x']):
    for j in range(dsp.dims.mapping['y']):
        for k in range(dsp.dims.mapping['sigma']):
            u = dsp['VxyX'].values[k,i,j]
            v = dsp['VxyY'].values[k,i,j]
            w = dsp['W'].values[k,i,j]
            # arr.InsertNextValue(u)
            # arr.InsertNextTuple((u,v,w))
            arr.SetTuple(id, [u,v,w])
            id += 1

#StructuredGridのPointDataに温度データを追加
structured_mesh.GetPointData().AddArray(arr)

writer = vtk.vtkXMLDataObjectWriter()
writer.SetFileName("tmp02.vts")
writer.SetInputData(structured_mesh)
writer.Update()

In [9]:
# writer = vtk.vtkXMLStructuredGridWriter()
writer =vtk.vtkDataSetWriter()
writer.SetFileName("tmp5.vtk")
writer.SetInputData(structured_mesh)
writer.Update()